# Q1 Data Icebreaker: Let’s break the ice with some data! (3 Marks)

In [ ]:
#imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Load the dataset
df = pd.read_csv('/path/to/TaxiData.csv')

```
Analysis on overall Dataset.
Provide info about columns and its Datatypes.
State missing values and other organic analysis.
```

### Q1.a
Categorize columns: classify columns as categorical, numerical, or mixed data types.

In [ ]:
# Check data shape, summary, ...

# Identify missing values
# Plot missing values (customize as needed)
# missing_values.plot(...)

# Any change of data type conversion required?


### Q1.b
Type Conversion: Convert data types where necessary to facilitate analysis, noting why certain columns are converted.

### Q1.c
Identify Missing Values: Check for missing values in each column and visualize them using a heatmap or bar chart. Discuss which columns have significant missing values.

### Q1.d
Handling Missing Values: Propose methods to handle missing data based on the column’s type (e.g., mean imputation for numerical,) and apply them where ever appropriate. Mention the method used and the column name on which it is applied.